In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns

In [2]:
data= pd.read_csv('20220422-Campa+¦as Etiquetadas updated.csv', encoding='latin-1')
data.head()

,Etiquetas,Mensaje
0,Comunidad,RT @Camiloparada: En el 2010 Pi?era nombr com...
1,Mineria,RT @Camiloparada: En el 2010 Pi?era nombr com...
2,Comunidad,RT @Camiloparada: En el 2010 Pi?era nombr com...
3,Mineria,RT @Camiloparada: En el 2010 Pi?era nombr com...
4,NaN,RT @Camiloparada: En el 2010 Pi?era nombr com...


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12930 entries, 0 to 12929
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Etiquetas  6802 non-null   object
 1   Mensaje    12897 non-null  object
dtypes: object(2)
memory usage: 202.2+ KB


In [4]:
# check description
data.describe().T

,count,unique,top,freq
Etiquetas,6802,59,pol pub,2511
Mensaje,12897,7067,Sin contenido,162


In [5]:
df = data.dropna()
df.shape

(6784, 2)

In [6]:
# check null values
df.isnull().sum()

Etiquetas    0
Mensaje      0
dtype: int64

In [7]:
print(data.shape)
print(df.shape)

(12930, 2)
(6784, 2)


In [8]:
df.to_csv('spanish_final.csv')

# Data Save in s3 Bucket

In [9]:
import boto3
import sagemaker

print(sagemaker.__version__)

session = sagemaker.Session()
bucket = session.default_bucket()

2.86.2


In [10]:
# Run this cell if you want to use the data you processed manually

prefix = 'sklearn-spanish-text-classification'

# s3_train_path = session.upload_data(path='spanish_final.csv', key_prefix=prefix + "/training")
# s3_output = 's3://{}/{}/output/'.format(bucket, prefix)

s3_train_path = 's3://sagemaker-us-east-1-122514903081/sklearn-spanish-text-classification/training/spanish_final.csv'
s3_output = 's3://sagemaker-us-east-1-122514903081/sklearn-spanish-text-classification/output/'


print(s3_train_path)
print(s3_output)

s3://sagemaker-us-east-1-122514903081/sklearn-spanish-text-classification/training/spanish_final.csv
s3://sagemaker-us-east-1-122514903081/sklearn-spanish-text-classification/output/


In [17]:
import sagemaker
from sagemaker.sklearn import SKLearn
role = sagemaker.get_execution_role()

env = {
    'SAGEMAKER_REQUIREMENTS': 'requirements.txt', # path relative to `source_dir` below.
}

hyperparameters={
                  'random-state': 16,
                  'test-size': 0.2
              }

sk = SKLearn(entry_point='spanish_text_classification.py',
             source_dir='src',
             role=role,
             framework_version='0.20.0',
             instance_count=1, 
             instance_type='ml.m5.xlarge',
             output_path=s3_output,
            hyperparameters=hyperparameters)

sk.fit(s3_train_path)

2022-06-30 13:20:36 Starting - Starting the training job...
2022-06-30 13:21:01 Starting - Preparing the instances for trainingProfilerReport-1656595236: InProgress
.........
2022-06-30 13:22:28 Downloading - Downloading input data...
2022-06-30 13:23:01 Training - Downloading the training image...
2022-06-30 13:23:21 Training - Training image download completed. Training in progress.2022-06-30 13:23:22,133 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2022-06-30 13:23:22,137 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-06-30 13:23:22,146 sagemaker_sklearn_container.training INFO     Invoking user training script.
2022-06-30 13:23:22,547 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/miniconda3/bin/python -m pip install -r requirements.txt
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.20.0
    Uninstalling scikit-learn-0.20.0:
  

# Model Deployment

In [ ]:
# predictor = sk.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

# Delete Endpoint

In [ ]:
# predictor.delete_endpoint()